In [1]:
import csv
import json
import re
import pyparsing

# Splitting

Initial splitting of the file into components

In [41]:
# Patterns
re_open_tag = r'\s*<(?P<name>[^/][a-z]+)(?P<attrs>[^>]*)>\s*'
re_close_tag = r'\s*</(?P<name>[a-z]+)\s*>\s*'


match = re.match(re_open_tag,'<config something="yeah" no>')
if match:
    print(match)
    print(match.groupdict())

<re.Match object; span=(0, 28), match='<config something="yeah" no>'>
{'name': 'config', 'attrs': ' something="yeah" no'}


In [32]:
match = re.match(re_close_tag,'</config>')
print(match)

<re.Match object; span=(0, 9), match='</config>'>


In [143]:
def parse_ccsv(filename):
    tags = {}
    csv_lines = []
    
    # First read csv content at the top
    with open(filename, 'r') as rfile:

        for line in csv.reader(rfile):
            open_tag_match = re.match(re_open_tag,line[0])
            if open_tag_match:
                print(f'exiting csv read, found tag in line:{line}')
                break

            else:
                csv_lines.append(line)

        if not bool(open_tag_match):
            return csv_lines, tags
        
            
        
        current_tag = open_tag_match.groupdict()['name']
        current_tag_contents = ''

        print('reading from readlines:')
        for line in rfile.readlines():
            if current_tag:
                close_tag_match = re.match(re_close_tag, line)
                if close_tag_match:
                    if close_tag_match.groupdict()['name'] == current_tag:
                        print(f'found closing tag for <{current_tag}>')
                    else:
                        raise ValueError(f'Incorrect closing tag found for <{current_tag}> tag')
                    
                    # Store the contents and clear the temporary variables
                    tags[current_tag] = current_tag_contents
                    current_tag_contents = ''
                    current_tag = None
                    open_tag_found = False
                    continue
                else:
                    current_tag_contents += line.strip()
            else:
                open_tag_match = re.match(re_open_tag,line)
                if open_tag_match:
                    current_tag = open_tag_match.groupdict()['name']
                    current_tag_contents = ''
                    continue
    
    return csv_lines, tags
            
parse_ccsv('examples/demo_v1.ccsv')

exiting csv read, found tag in line:['<config>']
reading from readlines:
found closing tag for <config>
found closing tag for <style>
found closing tag for <something>


([['item', 'cost', 'quantity', 'total'],
  ['chalk', '3.50', '2', '=cost*quantity'],
  ['cheese', '1.99', '10', '=cost*quantity'],
  ['Total{boldface}', '', 'sum(1:', 'total)']],
 {'config': '{"_version": 1.1,"_name": "mySheet"}',
  'style': 'th{background: blue;}.boldface{fontweight: 500;}',
  'something': 'content'})

In [144]:
csv_lines, tags = parse_ccsv('examples/demo_v1.ccsv')
csv_lines,tags


exiting csv read, found tag in line:['<config>']
reading from readlines:
found closing tag for <config>
found closing tag for <style>
found closing tag for <something>


([['item', 'cost', 'quantity', 'total'],
  ['chalk', '3.50', '2', '=cost*quantity'],
  ['cheese', '1.99', '10', '=cost*quantity'],
  ['Total{boldface}', '', 'sum(1:', 'total)']],
 {'config': '{"_version": 1.1,"_name": "mySheet"}',
  'style': 'th{background: blue;}.boldface{fontweight: 500;}',
  'something': 'content'})

In [54]:
tags['style']

'th{background: blue;}.boldface{fontweight: 500;}'

In [50]:
json.loads(tags['config'])

{'_version': 1.1, '_name': 'mySheet'}

# Parse calculations

Parse the math calculations

Examples:
- = 2+2
- = ['quantity'] * ['cost'] * 3
- = SUM([3:4,'total'])
- = SUM([3:.,'total']) # 3 to here?
- = SUM([3:,'total']) # 3 to the end


Todo:
1. Define a vector Class?
2. Define grammar and parsing
    - Do a simple parser
    - Look at using pyparsing

In [2]:
def simple_parser(cell_string):
    operators = {
        '*': lambda x,y:x*y,
        '+': lambda x,y:x+y,
        '-': lambda x,y:x-y,
        '/': lambda x,y:x/y
    }
    
    parts = []
    current_col, current_num = "", ""
    reading_num, reading_col = False, False
    
    for char in cell_string:
        
        if reading_num:
            if char.isdigit() or char=='.':
                current_num += char
                continue
            else:
                parts.append(current_num)
                current_num = ''
                reading_num=False
                
        # If currently reading a col
        if reading_col:
            if char.isalpha() or char=='_':
                current_col += char
                continue
                
            else:
                parts.append(current_col)
                current_col = ''
                reading_col=False
        
        if char.isalpha() or char=='_':
            current_col += char
            reading_col=True
            
        elif char in operators:
            parts.append(char)
            
        elif char.isdigit() or char=='.':
            current_num += char
            reading_num=True
    
    # Closing out 
    if reading_num:
        parts.append(current_num)
        
    elif reading_col:
        parts.append(current_col)
        
    return parts
        
simple_parser("total*3.12+7 - something ")

['total', '*', '3.12', '+', '7', '-', 'something']

In [111]:
csv_lines

[['item', 'cost', 'quantity', 'total'],
 ['chalk', '3.50', '2', '=cost*quantity'],
 ['cheese', '1.99', '1=cost*quantity'],
 ['Total{boldface}', '', 'sum(1:', 'total)']]

In [120]:
dict( (y,x) for x,y in enumerate(csv_lines[0]))

{'item': 0, 'cost': 1, 'quantity': 2, 'total': 3}

In [145]:
def compute_csv(lines):

    hdict = dict( (y,x) for x,y in enumerate(csv_lines[0]))

    for line in csv_lines:
        new_line = []
        changed=False
        for cell in line:
            if cell.startswith('='):
                parsed=simple_parser(cell.lstrip('= '))
                mult_ind = parsed.index('*')
                if mult_ind:
                    val1 = float(line[hdict[parsed[mult_ind-1]]])
                    val2 = float(line[hdict[parsed[mult_ind+1]]])
                    val = val1 * val2
                    changed=True
                    new_line.append(val)
            else:
                new_line.append(cell)
        
        print(f"{line}  -{'*' if changed else '-'}-> {new_line}")

compute_csv(csv_lines)


['item', 'cost', 'quantity', 'total']  ---> ['item', 'cost', 'quantity', 'total']
['chalk', '3.50', '2', '=cost*quantity']  -*-> ['chalk', '3.50', '2', 7.0]
['cheese', '1.99', '10', '=cost*quantity']  -*-> ['cheese', '1.99', '10', 19.9]
['Total{boldface}', '', 'sum(1:', 'total)']  ---> ['Total{boldface}', '', 'sum(1:', 'total)']


In [83]:
class String(object):
    def __init__(self, result):
        self.value = result[0]

    def generate(self):
        return "'{}'".format(self.value)

class Number(object):
    def __init__(self, result):
        self.value = result[0]

    def generate(self):
        return self.value

class Identifier(object):
    def __init__(self, result):
        self.name = result[0]

    def generate(self):
        return self.name

class Condition(object):
    def __init__(self, tokens):
        self.identifier = tokens[0][0]
        self.op = tokens[0][1]
        self.rval = tokens[0][2]

    def generate(self):
        return " ".join((self.identifier.generate(), self.op, self.rval.generate()))

lparen = pp.Suppress("(")
rparen = pp.Suppress(")")

and_ = pp.Literal("AND")
or_ = pp.Literal("OR")

operator = pp.oneOf(("=", "!=", ">", ">=", "<", "<="))

alphaword = pp.Word(pp.alphanums + "_")
string = pp.QuotedString(quoteChar="'").setParseAction(String)

number = (
 pp.Word(pp.nums) + pp.Optional("." + pp.OneOrMore(pp.Word(pp.nums)))
).setParseAction(Number)

identifier = alphaword.setParseAction(
 Identifier
)

condition = pp.Group(
 identifier + (operator + (string | number))
).setParseAction(Condition)

In [84]:
number.parseString('1.32')

([<__main__.Number object at 0x7f3fbcf25250>], {})

In [91]:
condition.parseString('y=7.3')[0].rval.value

'7'

# Yaml approach

In [32]:
!cat examples/demo_v3_yaml.ccsv

col1,col2
a,b,7
d,e,9


In [6]:
ypath = 'examples/demo_v3_yaml.ccsv'
ystring = open(ypath).read()

In [73]:
def file_splitter(filename):
    ''' 
    Split the file into csv and config
    
    Returns:
        csv_lines (list): list of strings where each string is a line of csv
        config (dict): the parsed config from the yaml
    
    '''
    reading_yaml_header = False
    yaml_line_count = 0
    
    csv_lines = []
    yaml_lines = []
    with open(filename, 'r') as rfile:
        for i, line in enumerate(csv.reader(rfile)):
            if i == 0 and line[0].startswith('---'):
                reading_yaml_header = True
                continue
                
            if reading_yaml_header:
                if line[0].startswith('---'):
                    reading_yaml_header = False
                    continue
                    
                else:
                    yaml_line_count += 1
            else:
                if len(line):
                    csv_lines.append(line)
    if yaml_line_count:
        with open(filename, 'r') as rfile:
            for i, line in enumerate(rfile):
                if i == 0:
                    continue 
                elif i > yaml_line_count: 
                    break
                else:
                    yaml_lines.append(line.rstrip())
        
    config = yaml.safe_load(StringIO("\n".join(yaml_lines)))
    return csv_lines, config

csv_lines, config= file_splitter(ypath)
csv_lines, config

([['item', 'cost', 'quantity', 'total'],
  ['chalk', '3.50', '2', '=cost*quantity'],
  ['cheese', '1.99', '1', '=cost*quantity']],
 {'name': 'test',
  'value': ['something1', 'something2,3,4'],
  'style': {'body': {'background': 'blue', 'color': 'lightgrey'}}})

In [21]:
from io import StringIO

In [10]:
import yaml

In [13]:
yaml.__version__

'6.0'

In [25]:
StringIO("".join(yaml_lines))

In [31]:
yaml.safe_load(StringIO("\n".join(yaml_lines)))

{'name': 'test',
 'value': ['something1', 'something2'],
 'style': {'.body': {'background': 'blue'}}}

In [ ]:
open()

In [39]:
import csv
with open('examples/demo_v3_yaml.ccsv', 'r') as rfile:
        for i, line in enumerate(csv.reader(rfile)):
            print(line)

['---']
['name: test']
['value:']
['  - something1']
['  - something2', '3', '4']
['style:']
['  ".body":']
['    background: blue']
['---']
['item', 'cost', 'quantity', 'total']
['chalk', '3.50', '2', '=cost*quantity']
['cheese', '1.99', '1', '=cost*quantity']


In [68]:
def compile_css(style_obj, indent=2):
    # Assume simple structure
    css_string = ''
    for selector, rules in style_obj.items():
        css_string += f"{selector} {{\n"
        for key,val in rules.items():
            css_string += f"{indent*' '} {key}: {val};\n"
        css_string += '}\n'
            
    return css_string
res = compile_css(config['style'])
print(res)
        

.body {
   background: blue;
   color: lightgrey;
}



In [64]:
from pathlib import Path

In [74]:
def build_table(csv_path):
    table_string = ''
    table_string += '<table>\n<tbody>\n'
    table_string += '<tbody>'
    with open(csv_path, 'r') as rfile:
        for i, row in enumerate(csv.reader(rfile)):
            table_string += '<tr>\n'
            tag = 'td' if i!=0 else 'th'
            for cell in row:
                table_string += f"<{tag}> {cell} </{tag}>"
            table_string += '</tr>\n'
            
        table_string += '</tbody>\n</table>'
    return table_string

def build_html(csv_path, config, outdir='examples'):
    csv_path = Path(csv_path)
    css_string = compile_css(config['style'])
    table_string = build_table(csv_path) 
    html_string = f'''
    <html>
      <head>
        <style>
        {css_string}
        </style>
      </head>
      <body>
        {table_string}
        </body>
    </html>
    '''
    if outdir:
        with open(f"{outdir}/{csv_path.stem}.html", 'w') as wfile:
            wfile.write(html_string)
    return html_string
    
print(build_html('examples/demo_v3_yaml.csv', config))


    <html>
      <head>
        <style>
        body {
   background: blue;
   color: lightgrey;
}

        </style>
      </head>
      <body>
        <table>
<tbody>
<tbody><tr>
<th> item </th><th> cost </th><th> quantity </th><th> total </th></tr>
<tr>
<td> chalk </td><td> 3.50 </td><td> 2 </td><td> 7.0 </td></tr>
<tr>
<td> cheese </td><td> 1.99 </td><td> 1 </td><td> 1.99 </td></tr>
</tbody>
</table>
        </body>
    </html>
    
